# Image registration for Rededge Rivers

As it is a sequoia orthomosaic, scaled from 0-to-1, we assume it is calibrated
* For the following Dataset: https://zenodo.org/records/7144071 
* Using code snippets from:

Output: `uint16` 0-65535 scaled .tif files with green, red, rededge, nir bands


In [4]:
import os
import rioxarray as rxr
import xarray as xr
import tifffile
from glob import glob
import numpy as np
from itertools import repeat

In [14]:
def load_image(path):
    return rxr.open_rasterio(path), path
    
def rescale_to_uint16(da):
    da = da.where(~np.isnan(da), 65535)
    rescaled = (da * 65535).clip(0, 65535).astype(np.uint16)
    
    return rescaled

def rescale_float32_to_uint16_xarray(xarr):
    # Mask NaNs and -1000 values
    xarr = xr.where(xarr == -1, 1, xarr)
    
    # Find min and max values ignoring NaNs
    min_val = 0
    max_val = 1
    
    # Normalize to [0, 1] ignoring NaNs
    normalized = (xarr - min_val) / (max_val - min_val)
    
    # Scale to [0, 65534]
    scaled = (normalized * 65535).astype(np.uint16)
    
    # Set NaNs and -1000 to 65535
    scaled = scaled.where(~xarr.isnull() & (xarr != -1000), 65535)
    
    return scaled
    
def create_4_channel_image(green, red,rededge, nir):
    """Create a 4-channel image (G, R, RE, NIR)."""
    return np.stack((green, red, rededge, nir), axis=-1)
    

def save_image(filename, image, metadata=None):
    """Save the image as a 4-channel TIFF using tifffile."""
    #tifffile.imwrite(filename, image, photometric='rgb',extratags=metadata[0],subifds = metadata[1])
    if metadata is not None:
        extratags = metadata[0]
        subifds = metadata[1]
        with tifffile.TiffWriter(filename) as tiff:
            tiff.write(
                image,
                photometric='rgb',
                extratags=extratags,
                subifds=subifds
            )
    else:
        with tifffile.TiffWriter(filename) as tiff:
            tiff.write(
                image,
                photometric='rgb')
            


def process_multispec_set(green_image_path, output_directory):

    base_name = os.path.basename(green_image_path).replace("green.data.tif", "")
    # Load the green band image (reference image)
    dirname = os.path.dirname(green_image_path)
    ###REDO WHEN THIS NEEDS REDOING
    green = load_image(green_image_path)
    red = load_image(base_name+"red.data.tif")
    rededge = load_image(base_name+"rededge.data.tif")
    nir = load_image(base_name+"nir.data.tif")
    print(nir)
    print(nir.min())
    print(nir.max())
    green = rescale_float32_to_uint16_xarray(green)
    red = rescale_float32_to_uint16_xarray(red)
    rededge = rescale_float32_to_uint16_xarray(rededge)
    nir = rescale_float32_to_uint16_xarray(nir)

    
     # Create a 4-channel image
    image_4ch = create_4_channel_image(green, red, rededge, nir)

    sensor = "REDEDGE"
    cal = "CAL"
    set_title = os.path.basename(os.path.normpath(output_directory))
    output_filename = f"{sensor}_{cal}_{set_title}_{base_name}.tif"
    output_path = os.path.join(output_directory, output_filename)
    os.makedirs(output_directory, exist_ok=True)


    # Save the 4-channel image
    save_image(output_path, image_4ch)

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def process_directory(input_dir, output_directory, n_threads= 1):
    """Process an entire directory with multiple sets of multispectral images."""
    input_dir = os.path.abspath(input_dir)
    os.makedirs(output_directory, exist_ok=True)

    # Find all green band images (reference images)
    green_images = glob(os.path.join(input_dir, "*MS_DLS*.TIF"))

    # Process sets of 4 multispectral images in parallel with a progress bar
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        list(tqdm(executor.map(process_multispec_set, green_images, repeat(output_directory)), total=len(green_images)))

    print("All images successfully processed.")

In [ ]:
@@@ ALL WRONG

In [15]:
folder = "../data/Rededge_rivers/Emmericher"
input_folder = folder
output_folder = "../data/processed/rededge_rivers_em/"
process_directory(input_folder, output_folder)

0it [00:00, ?it/s]

All images successfully processed.


In [16]:
folder = "../data/Rededge_rivers/Kuehkopf"
input_folder = folder
output_folder = "../data/processed/rededge_rivers_kk/"
process_directory(input_folder, output_folder)

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

<xarray.DataArray ()> Size: 4B
array(-1., dtype=float32)
Coordinates:
    band         int64 8B 5
    spatial_ref  int64 8B 0


C:\Users\judoj\mambaforge\envs\msdata\Lib\site-packages\xarray\core\duck_array_ops.py:234: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
 33%|████████████████████████████                                                        | 1/3 [00:01<00:02,  1.43s/it]

<xarray.DataArray ()> Size: 4B
array(-1., dtype=float32)
Coordinates:
    band         int64 8B 5
    spatial_ref  int64 8B 0


C:\Users\judoj\mambaforge\envs\msdata\Lib\site-packages\xarray\core\duck_array_ops.py:234: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
 67%|████████████████████████████████████████████████████████                            | 2/3 [00:02<00:01,  1.41s/it]

<xarray.DataArray ()> Size: 4B
array(-0.33537203, dtype=float32)
Coordinates:
    band         int64 8B 5
    spatial_ref  int64 8B 0


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.36s/it]

All images successfully processed.


In [17]:
folder = "../data/Rededge_rivers/Laubenheim"
input_folder = folder
output_folder = "../data/processed/rededge_rivers_lh/"
process_directory(input_folder, output_folder)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

<xarray.DataArray ()> Size: 8B
array(0.)
Coordinates:
    band         int64 8B 5
    spatial_ref  int64 8B 0


C:\Users\judoj\mambaforge\envs\msdata\Lib\site-packages\xarray\core\duck_array_ops.py:234: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
 50%|██████████████████████████████████████████                                          | 1/2 [00:00<00:00,  1.83it/s]

<xarray.DataArray ()> Size: 8B
array(-1.)
Coordinates:
    band         int64 8B 5
    spatial_ref  int64 8B 0


C:\Users\judoj\mambaforge\envs\msdata\Lib\site-packages\xarray\core\duck_array_ops.py:234: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
C:\Users\judoj\mambaforge\envs\msdata\Lib\site-packages\xarray\core\duck_array_ops.py:234: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.57it/s]

All images successfully processed.


In [18]:
folder = "../data/Rededge_rivers/Niederwerth_s"
input_folder = folder
output_folder = "../data/processed/rededge_rivers_ns/"
process_directory(input_folder, output_folder)

0it [00:00, ?it/s]

All images successfully processed.


In [19]:
folder = "../data/Rededge_rivers/Niederwerth_w"
input_folder = folder
output_folder = "../data/processed/rededge_rivers_nw/"
process_directory(input_folder, output_folder)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

<xarray.DataArray ()> Size: 8B
array(-1.)
Coordinates:
    band         int64 8B 5
    spatial_ref  int64 8B 0


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.81it/s]

All images successfully processed.


In [20]:
folder = "../data/Rededge_rivers/Nonnenwerth"
input_folder = folder
output_folder = "../data/processed/rededge_rivers_nw"
process_directory(input_folder, output_folder)

0it [00:00, ?it/s]

All images successfully processed.


In [21]:
folder = "../data/Rededge_rivers/Reitwein"
input_folder = folder
output_folder = "../data/processed/rededge_rivers_rw/"
process_directory(input_folder, output_folder)

0it [00:00, ?it/s]

All images successfully processed.
